### CMU Summaries

In [1]:
import tarfile
my_tar = tarfile.open('/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/MovieSummaries.tar.gz')
my_tar.extractall('./CMU_Summaries')
my_tar.close()

In [1]:
with open('/content/CMU_Summaries/MovieSummaries/README.txt','r') as f:
  readme_data = f.read()
# print(readme_data)

In [2]:
import pandas as pd
df_movie_meta = pd.read_table('/content/CMU_Summaries/MovieSummaries/movie.metadata.tsv', header=None, names = ['Wikipedia movie ID','Freebase movie ID','Movie name','Movie release date','Movie box office revenue','Movie runtime','Movie language','Movie countries','Movie genres'])

In [ ]:
df_movie_meta

In [3]:
df_charac_meta = pd.read_table('/content/CMU_Summaries/MovieSummaries/character.metadata.tsv',header=None, names = ['Wikipedia movie ID', 'Freebase movie ID','Movie release date','Character name','Actor date of birth','Actor gender','Actor height','Actor ethnicity','Actor name','Actor age at movie release','Freebase character/actor map ID','Freebase character ID','Freebase actor ID'])

In [ ]:
df_charac_meta

In [4]:
with open('/content/CMU_Summaries/MovieSummaries/plot_summaries.txt','r') as f:
  summaries = f.read()

In [5]:
summaries = summaries.split('\n')

In [ ]:
summaries

In [6]:
df_summaries = pd.DataFrame(columns=['Wikipedia movie ID', 'Summary'])

In [7]:
df_summaries['Summary'] = summaries

In [8]:
def sep_id(x):
  return x.split('\t')[0]
def sep_summary(x):
  try:
    return x.split('\t')[1]
  except:
    return None

In [9]:
df_summaries['Wikipedia movie ID']  = df_summaries['Summary'].apply(sep_id)
df_summaries['Summary'] = df_summaries['Summary'].apply(sep_summary)

In [10]:
df_summaries.dropna(inplace=True)

In [11]:
df_summaries.reset_index(inplace=True, drop=True)

In [ ]:
#Combining movie summaries with other information - movie and character metadata 

In [12]:
df_summaries = df_summaries.assign(Genre="", Movie_Name="",Characters="")

In [ ]:
df_summaries

In [13]:
labels = []
for idx, row in df_summaries.iterrows():
  # for idx2, row2 in df_movie_meta.iterrows():
    #Get index and then assign values, avoid the for loop and if condition (do the same for characters)
    # if row2['Wikipedia movie ID'] == row['Wikipedia movie ID']:
      # df_summaries['Movie_Name'].iloc[idx] = row2['Movie name']
      # df_summaries['Genre'].iloc[idx] = row2['Movie genres']
      # break
  try: 
    index = df_movie_meta['Wikipedia movie ID'].values.tolist().index(int(row['Wikipedia movie ID']))
    df_summaries['Movie_Name'].iloc[idx] = df_movie_meta['Movie name'].iloc[index]
    df_summaries['Genre'].iloc[idx] = df_movie_meta['Movie genres'].iloc[index]
  except:
    labels.append(idx)

In [ ]:
labels

In [ ]:
len(labels)

In [ ]:
df_summaries

In [14]:
df_summaries.drop(index=labels, axis=0, inplace=True)
df_summaries.reset_index(drop=True, inplace=True)

In [33]:
# Extract Genre 
import json
def get_genre(x):
  return ', '.join(json.loads(x).values())
df_summaries['Genre'] = df_summaries['Genre'].apply(get_genre)

In [15]:
labels=[]
for idx, row in df_summaries.iterrows():
  index=-1
  check = True
  # for idx2, row2 in df_charac_meta.iterrows():
  #   if row2['Wikipedia movie ID'] == row['Wikipedia movie ID']:
  #     df_summaries['Characters'].iloc[idx] = df_summaries['Characters'].iloc[idx]+row2['Character name']
  print()
  print(row['Movie_Name'], end=' ')
  try:  
    while True:
      index = df_charac_meta['Wikipedia movie ID'].values.tolist()[index+1:].index(int(row['Wikipedia movie ID']))+index+1
      df_summaries['Characters'].iloc[idx] = df_summaries['Characters'].iloc[idx] + ", " + df_charac_meta['Character name'].iloc[index]
      print(df_charac_meta['Character name'].iloc[index], end=' ')
      check = False
  except:
    if check:
      labels.append(idx)

In [ ]:
df_summaries

In [17]:
len(labels)

28289

In [20]:
# Take characters from BookNLP instead of the corpus
df_summaries.drop(['Characters'], axis=1, inplace=True)
df_summaries = df_summaries.assign(Characters="")

In [35]:
# Save dataframe
df_summaries.to_csv('/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/CMU_Movie_Dataset', index=False)

### CMU Book Summaries

In [36]:
import tarfile
my_tar = tarfile.open('/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/booksummaries.tar.gz')
my_tar.extractall('./CMU_Book_Summaries')
my_tar.close()

In [ ]:
with open('/content/CMU_Book_Summaries/booksummaries/README','r') as f:
  readme_data = f.read()
print(readme_data)

In [42]:
columns = ['Wikipedia article ID','Freebase ID','Book title','Author','Publication date','Book genres','Plot summary']
df_summaries = pd.read_table('/content/CMU_Book_Summaries/booksummaries/booksummaries.txt', header=None, names=columns)

In [ ]:
df_summaries

In [44]:
df_summaries.drop(['Freebase ID','Author','Publication date'], axis=1, inplace=True)

In [50]:
df_summaries.dropna(axis=0, inplace=True)

In [57]:
import json
def extract_genres(x):
  return ", ".join(json.loads(x).values())

In [58]:
df_summaries['Book genres'] = df_summaries['Book genres'].apply(extract_genres)

In [60]:
df_summaries.to_csv('/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/CMU_Book_Dataset', index=False)